Temporary notebook to run and manually test the OSM feature generation function.

PHL cluster-level data is taken from [GDrive](https://drive.google.com/drive/u/0/folders/1N9vSFX05bDWGolfsRjTTBUxGtuVP-77s).

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../../../")

import uuid


import geopandas as gpd
import pandas as pd

from povertymapping import settings, osm
from povertymapping.osm import OsmDataManager

Load in temporary data for PHL clusters

In [ ]:
def load_cluster_gdf(path):
    df = pd.read_csv(path)

    # Some of the coordinates in the data are invalid. This filters them out.
    df = df[(df.longitude>0)&(df.latitude>0)]

    # Create a GeoDataFrame from the longitude, latitude columns.
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="epsg:4326")

    print(f"There are {len(gdf):,} clusters.")

    return gdf

In [ ]:
GROUND_TRUTH_CSV = settings.DATA_DIR/"phl_dhs_cluster_level.csv"
gdf = load_cluster_gdf(GROUND_TRUTH_CSV)

Try the OSM feature generation function.

In [ ]:
%%time
gdf_with_features = osm.add_osm_features(gdf, "philippines", cache_dir=settings.CACHE_DIR)
gdf_with_features.head()

Sanity check that the internal helper function can successfully download and cache OSM data for the DHS countries.

In [ ]:
countries = [
    "philippines", "cambodia", "east-timor", "myanmar"
]

for country in countries:
    osm.download_osm_country_data(country, cache_dir=settings.CACHE_DIR)

# Roads

In [ ]:
GROUND_TRUTH_CSV = settings.DATA_DIR/"phl_dhs_cluster_level.csv"
aoi = load_cluster_gdf(GROUND_TRUTH_CSV)
aoi["geometry"] = aoi.to_crs("EPSG:3857").geometry.buffer(2000).to_crs("epsg:4326")

In [ ]:
osm_data_manager = OsmDataManager(cache_dir=settings.CACHE_DIR)

In [ ]:

country = "philippines"
aoi_with_road_stats = osm.add_osm_poi_features(aoi, country, osm_data_manager)
aoi_with_road_stats = osm.add_osm_road_features(aoi, country, osm_data_manager)
aoi_with_road_stats.explore()